<h1>Exploration and Clustering of Toronto Neighborhoods</h1>

<h3>Procedure done is similar to the New York/Manhattan lab and functions used are the same ones.</h3>

In [ ]:
import pandas as pd
TorontoNeigh_loc = pd.read_pickle("TorontoNeigh_loc.pkl")

In [2]:
TorontoNeigh_loc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
#Retrieve subset of Neighborhood with Boroughs' name containing Toronto
CentralTorontoNeigh = TorontoNeigh_loc[TorontoNeigh_loc['Borough'].str.contains("Tor")].reset_index(drop=True)

In [4]:
CentralTorontoNeigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [5]:
#Import modules for mapping and dataframe processing

import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium 

# library to handle requests
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [6]:
#Creating initial map of Toronto, Ontario.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


In [8]:
# add markers to map (Boroughs of Toronto)
for lat, lng, borough, neighborhood in zip(CentralTorontoNeigh['Latitude'], CentralTorontoNeigh['Longitude'], CentralTorontoNeigh['Borough'], CentralTorontoNeigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

In [9]:
#Verification of FourSuare Credentials
CLIENT_ID = 'IMAHLL44LVEWQJF2SCZVW3WEHM3OSXAKZNJB2OFX3CIDAUZ1' # your Foursquare ID
CLIENT_SECRET = 'UFPOFM1CWNMGYDTDH34JL03Q5ITSX2VLGLBZJ4BZ0XFH4LBS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: IMAHLL44LVEWQJF2SCZVW3WEHM3OSXAKZNJB2OFX3CIDAUZ1
CLIENT_SECRET:UFPOFM1CWNMGYDTDH34JL03Q5ITSX2VLGLBZJ4BZ0XFH4LBS


In [10]:
#Processing a smaller sample, East Toronto from the Toronto Boroughs.
EastToronto_data = CentralTorontoNeigh[CentralTorontoNeigh['Borough'] == 'East Toronto'].reset_index(drop=True)
EastToronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [45]:
EastToronto_data.shape

(5, 5)

In [12]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = EastToronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = EastToronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = EastToronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [13]:
#Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

addressET = 'The Beach, ON'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(addressET)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

print(latitude)
print(longitude)


The geograpical coordinate of Toronto are 43.6710244, -79.296712.
43.6710244
-79.296712


In [14]:
radius = 500
LIMIT = 100
search_query = 'fast'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
   CLIENT_ID,
   CLIENT_SECRET,
   latitude,
   longitude,
   VERSION,
   radius,
   LIMIT)

url # display url

'https://api.foursquare.com/v2/venues/explore?&client_id=IMAHLL44LVEWQJF2SCZVW3WEHM3OSXAKZNJB2OFX3CIDAUZ1&client_secret=UFPOFM1CWNMGYDTDH34JL03Q5ITSX2VLGLBZJ4BZ0XFH4LBS&ll=43.6710244,-79.296712&v=20180605&radius=500&limit=100'

In [15]:
results = requests.get(url).json()
#results

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
   try:
       categories_list = row['categories']
   except:
       categories_list = row['venue.categories']
      
   if len(categories_list) == 0:
       return None
   else:
       return categories_list[0]['name']


In [17]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Kew Gardens,Park,43.669038,-79.298538
1,Mastermind Toys,Toy / Game Store,43.671453,-79.293971
2,Kew-Balmy Beach,Beach,43.667372,-79.295312
3,LCBO,Liquor Store,43.669993,-79.300383
4,Yumei Sushi,Japanese Restaurant,43.671108,-79.295064


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


In [ ]:
#Explore Neighborhoods in The Beach

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [20]:
EastToronto_venues = getNearbyVenues(names=EastToronto_data['Neighborhood'],
                                  latitudes=EastToronto_data['Latitude'],
                                  longitudes=EastToronto_data['Longitude']
                                 )


The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [21]:
print(EastToronto_venues.shape)
EastToronto_venues.head()

(121, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,My Bbq,43.676881,-79.289286,BBQ Joint
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [22]:
# one hot encoding
EastToronto_onehot = pd.get_dummies(EastToronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
EastToronto_onehot['Neighborhood'] = EastToronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [EastToronto_onehot.columns[-1]] + list(EastToronto_onehot.columns[:-1])
EastToronto_onehot = EastToronto_onehot[fixed_columns]

EastToronto_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Neighborhood,New American Restaurant,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
EastToronto_onehot.shape

(121, 66)

In [24]:
EastToronto_grouped = EastToronto_onehot.groupby('Neighborhood').mean().reset_index()
EastToronto_grouped


,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,New American Restaurant,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.00000,0.00000,0.0625,0.00,0.00000,0.000,0.000,0.00,0.00000,0.06250,0.00000,0.00,0.0625,0.0,0.00000,0.000,0.000,0.000,0.000000,0.000,0.0625,0.000,0.00000,0.000,0.00000,0.00000,0.0625,0.0625,0.00,0.000,0.00000,0.00000,0.0625,0.0625,0.000,0.000000,0.00000,0.00,0.000,0.000000,0.00000,0.00000,0.00000,0.000,0.125,0.00000,0.000,0.00,0.000,0.000,0.0625,0.00,0.06250,0.00000,0.0625,0.06250,0.000,0.000,0.0625,0.0625,0.00000,0.000,0.00,0.00,0.00000
1,Studio District,0.02500,0.05000,0.0000,0.00,0.05000,0.025,0.025,0.00,0.02500,0.02500,0.00000,0.00,0.0000,0.1,0.00000,0.025,0.025,0.025,0.075000,0.025,0.0000,0.025,0.00000,0.025,0.00000,0.02500,0.0000,0.0000,0.00,0.025,0.00000,0.00000,0.0000,0.0000,0.025,0.000000,0.00000,0.00,0.025,0.025000,0.00000,0.05000,0.02500,0.025,0.000,0.00000,0.025,0.00,0.025,0.025,0.0250,0.00,0.00000,0.00000,0.0000,0.00000,0.025,0.025,0.0250,0.0000,0.00000,0.025,0.00,0.00,0.00000
2,The Beaches,0.00000,0.00000,0.0000,0.25,0.00000,0.000,0.000,0.00,0.00000,0.00000,0.00000,0.00,0.0000,0.0,0.00000,0.000,0.000,0.000,0.250000,0.000,0.0000,0.000,0.00000,0.000,0.00000,0.00000,0.0000,0.0000,0.00,0.000,0.00000,0.00000,0.0000,0.0000,0.000,0.000000,0.00000,0.00,0.000,0.000000,0.00000,0.00000,0.00000,0.000,0.000,0.00000,0.000,0.00,0.000,0.000,0.0000,0.00,0.00000,0.25000,0.0000,0.00000,0.000,0.000,0.0000,0.0000,0.00000,0.000,0.00,0.00,0.00000
3,"The Beaches West,India Bazaar",0.00000,0.00000,0.0000,0.00,0.00000,0.000,0.000,0.05,0.00000,0.05000,0.00000,0.05,0.0500,0.0,0.00000,0.000,0.000,0.000,0.050000,0.000,0.0000,0.000,0.00000,0.000,0.00000,0.00000,0.0000,0.0500,0.05,0.000,0.00000,0.00000,0.0000,0.0000,0.000,0.000000,0.00000,0.05,0.000,0.050000,0.00000,0.05000,0.00000,0.000,0.000,0.05000,0.000,0.05,0.000,0.000,0.1000,0.05,0.05000,0.05000,0.0000,0.00000,0.050,0.000,0.0000,0.0000,0.00000,0.000,0.05,0.05,0.00000
4,"The Danforth West,Riverdale",0.02439,0.02439,0.0000,0.00,0.02439,0.000,0.000,0.00,0.04878,0.02439,0.02439,0.00,0.0000,0.0,0.02439,0.000,0.000,0.000,0.097561,0.000,0.0000,0.000,0.02439,0.000,0.02439,0.02439,0.0000,0.0000,0.00,0.000,0.02439,0.02439,0.0000,0.0000,0.000,0.243902,0.02439,0.00,0.000,0.073171,0.02439,0.04878,0.02439,0.000,0.000,0.02439,0.000,0.00,0.000,0.000,0.0000,0.00,0.02439,0.02439,0.0000,0.02439,0.000,0.000,0.0000,0.0000,0.02439,0.000,0.00,0.00,0.02439


In [25]:
EastToronto_grouped.shape

(5, 66)

In [26]:
num_top_venues = 5

for hood in EastToronto_grouped['Neighborhood']:
   print("----"+hood+"----")
   temp = EastToronto_grouped[EastToronto_grouped['Neighborhood'] == hood].T.reset_index()
   temp.columns = ['venue','freq']
   temp = temp.iloc[1:]
   temp['freq'] = temp['freq'].astype(float)
   temp = temp.round({'freq': 2})
   print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
   print('\n')


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1                Garden  0.06
2         Burrito Place  0.06
3         Garden Center  0.06
4  Fast Food Restaurant  0.06


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Beaches----
                       venue  freq
0                  BBQ Joint  0.25
1                        Pub  0.25
2                Coffee Shop  0.25
3                Yoga Studio  0.00
4  Latin American Restaurant  0.00


----The Beaches West,India Bazaar----
                venue  freq
0                Park  0.10
1         Coffee Shop  0.05
2      Sandwich Place  0.05
3      Ice Cream Shop  0.05
4  Italian Restaurant  0.05


----The Danforth West,Riverdale----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10


In [27]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
   row_categories = row.iloc[1:]
   row_categories_sorted = row_categories.sort_values(ascending=False)
  
   return row_categories_sorted.index.values[0:num_top_venues]


In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
   try:
       columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
   except:
       columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = EastToronto_grouped['Neighborhood']

for ind in np.arange(EastToronto_grouped.shape[0]):
   neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(EastToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Recording Studio,Garden Center,Comic Shop,Burrito Place,Fast Food Restaurant,Brewery,Park,Pizza Place
1,Studio District,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Gastropub,Juice Bar,Ice Cream Shop,Gym / Fitness Center,Cheese Shop
2,The Beaches,Coffee Shop,BBQ Joint,Pub,Trail,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop
3,"The Beaches West,India Bazaar",Park,Liquor Store,Pub,Gym,Italian Restaurant,Burrito Place,Burger Joint,Coffee Shop,Movie Theater,Brewery
4,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store


In [ ]:
#Cluster Neighborhoods

In [34]:
# import k-means from clustering stage
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5

EastToronto_grouped_clustering = EastToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(EastToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

#array([2, 2, 0, 2, 0, 2, 2, 3, 0, 2], dtype=int32)

#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

EastToronto_merged =EastToronto_data

# merge East Toronto_grouped with East Toronto_data to add latitude/longitude for each neighborhood
EastToronto_merged = EastToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

EastToronto_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,BBQ Joint,Pub,Trail,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Park,Liquor Store,Pub,Gym,Italian Restaurant,Burrito Place,Burger Joint,Coffee Shop,Movie Theater,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Gastropub,Juice Bar,Ice Cream Shop,Gym / Fitness Center,Cheese Shop
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,3,Light Rail Station,Garden,Recording Studio,Garden Center,Comic Shop,Burrito Place,Fast Food Restaurant,Brewery,Park,Pizza Place


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(EastToronto_merged['Latitude'], EastToronto_merged['Longitude'], EastToronto_merged['Neighborhood'], EastToronto_merged['Cluster Labels']):
   label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
   folium.CircleMarker(
       [lat, lon],
       radius=5,
       popup=label,
       color=rainbow[cluster-1],
       fill=True,
       fill_color=rainbow[cluster-1],
       fill_opacity=0.7).add_to(map_clusters)
     
map_clusters



In [ ]:
#Examine Clusters

In [37]:
#Cluster 0
EastToronto_merged.loc[EastToronto_merged['Cluster Labels'] == 0, EastToronto_merged.columns[[1] + list(range(5, EastToronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Grocery Store,Furniture / Home Store,Fruit & Vegetable Store


In [38]:
#Cluster 1
EastToronto_merged.loc[EastToronto_merged['Cluster Labels'] == 1, EastToronto_merged.columns[[1] + list(range(5, EastToronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,1,Park,Liquor Store,Pub,Gym,Italian Restaurant,Burrito Place,Burger Joint,Coffee Shop,Movie Theater,Brewery


In [42]:
#Cluster 2
EastToronto_merged.loc[EastToronto_merged['Cluster Labels'] == 2, EastToronto_merged.columns[[1] + list(range(5, EastToronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Coffee Shop,BBQ Joint,Pub,Trail,Coworking Space,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Dessert Shop


In [43]:
#Cluster 3
EastToronto_merged.loc[EastToronto_merged['Cluster Labels'] == 3, EastToronto_merged.columns[[1] + list(range(5, EastToronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Light Rail Station,Garden,Recording Studio,Garden Center,Comic Shop,Burrito Place,Fast Food Restaurant,Brewery,Park,Pizza Place


In [44]:
#Cluster 4
EastToronto_merged.loc[EastToronto_merged['Cluster Labels'] == 4, EastToronto_merged.columns[[1] + list(range(5, EastToronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Gastropub,Juice Bar,Ice Cream Shop,Gym / Fitness Center,Cheese Shop
